# 002 Data Preparation 1
* This notebook includes scripts to identify pings on highways or major roads.
* Gepspatial processing is included.

In [1]:
import pandas as pd
import numpy as np
from numpy import arange
import datetime
import matplotlib.pyplot as plt
import csv
import time
import geopandas as gpd
import shapefile
from rtree import index
from shapely.geometry import Polygon, Point
from pyproj import Proj, transform
import multiprocessing as mp
import tqdm
import os 
import json
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

os.system("tasket -p 0xff%d" % os.getpid())

32512

In [2]:
# Check package versions
import types 
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

import pkg_resources
root_packages = [i.split('.', 1)[0] for i in list(imports())] 
for m in pkg_resources.working_set:
    if m.project_name.lower() in root_packages:
        print (m.project_name, m.version)

('tqdm', '4.28.1')
('Rtree', '0.8.3')
('pandas', '0.24.0')
('numpy', '1.15.4')
('matplotlib', '2.2.3')
('geopandas', '0.4.0')


In [ ]:
def convert_bytes(num):
    """this function will convert bytes to MB, GB, etc"""
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0
        
def file_size(file_path):
    """this function will return the file size"""
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

# Excluding pings on highways
## 1. Load highway buffer shape

In [ ]:
highway = gpd.GeoDataFrame.from_file('../../data/highway_buffers.shp')
highway = highway.to_crs({'init' :'epsg:4326'})
print (highway.crs)

## 2. Load all pings (x,y) and create function
* to filter pings on highways or major roads
* using multiprocessing
* labeled datasets (1=highway) are saved in the directory

In [ ]:
fp = '../../data/houston_aug_sep_2017.csv'
df = pd.read_csv(fp)

In [ ]:
start = time.clock()
print (start)

# print range(10001,353220002,10000)
# num_rows = 353216526

def processInput(input_):
        x,y = input_
        return Point(x,y).within(highway.geometry[0])


for subset in range(10001,353220000,10000):
    #print subset
    tic = time.clock()

    inputs = list(zip(df.iloc[subset-10001:subset-10001,2],df.iloc[subset-10001:subset-10001,3]))

    pool = mp.Pool(processes = (mp.cpu_count() - 1))

    results = pool.map(processInput, inputs)
    pool.close()
    pool.join()

    toc = time.clock()
    tictoc1 = toc-tic
      
    # write a text file (save as txt file)
    tic = time.clock()
    with open('../../data/result_excluding_pings_highway/pings_highway_%s.txt' %(subset-1), 'w') as filehandle:
        json.dump(results, filehandle)
    toc = time.clock()
    tictoc2 = toc-tic
    
    print subset-10000, "to", subset-1, "rows ---",tictoc1, "seconds (time to run) ", tictoc2, "seconds (time to save)", file_size('result_excluding_pings_highway/pings_highway_%s.txt' %(subset-1))


done = time.clock()    
print (done)
print ("Total time is ", (done-start))